In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Basic Preprocessing

In [10]:
df = pd.read_csv("db_sfc_prsr_dd.csv")
df

,a.tma,a.stn_id,a.avg_pa,a.avg_pa_qcflg,a.avg_pa_crob,a.max_pa,a.max_pa_qcflg,a.max_pa_crob,a.max_pa_hrmt,a.min_pa,...,a.avg_pa_srv_yn,a.max_pa_srv_yn,a.min_pa_srv_yn,a.avg_ps_srv_yn,a.max_ps_srv_yn,a.min_ps_srv_yn,a.max_pa_hrmt_srv_yn,a.min_pa_hrmt_srv_yn,a.max_ps_hrmt_srv_yn,a.min_ps_hrmt_srv_yn
0,2011-01-01 00:00:00.0,79,999.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-01-01 00:00:00.0,80,-99.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011-01-01 00:00:00.0,81,1018.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-01-02 00:00:00.0,79,1005.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-01-02 00:00:00.0,80,-99.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10529,2016-03-27 00:00:00.0,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10530,2016-03-28 00:00:00.0,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10531,2016-03-29 00:00:00.0,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10532,2016-03-30 00:00:00.0,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def preprocess(df, dfname = "", to_csv = False):
    if type(df) == str: #df대신 df 경로 및 파일명 입력시
        df = pd.read_csv(df, encoding = "utf-8-sig")
    if df.columns[0] == "Unnamed: 0":
        df = df.loc[:, 1:]
    df.columns = [x.replace("a.", "") for x in df.columns] #colnames
    df["yyyymmdd"] = df[df.columns[0]].apply(lambda x: int(x.split(" ")[0].replace("-", "")))
    df = df.loc[:, df.isnull().mean() != 1]
    if to_csv == True:
        df.to_csv(dfname + ".csv", index = False, encoding = "utf-8-sig")
    return df

def daily(df, back, dfname = "", to_csv = False):
    if type(df) == str: #df대신 df 경로 및 파일명 입력시
        df = pd.read_csv(df, encoding = "utf-8-sig")
    final = pd.DataFrame()
    for s in df["stn_id"].unique():
        print(s, " is processing")
        ss = pd.merge(back, df[df["stn_id"] == s], how = "left", on = "yyyymmdd")
        ss["stn_id"] = s
        final = pd.concat([final, ss], axis = 0)
    if to_csv == True:
        final.to_csv(dfname + ".csv", encoding = "utf-8-sig", index = False)
    return final

def after_daily(df, area, dfname = "", to_csv = False):
    if type(df) == str:
        df = pd.read_csv(df)
    if "isnull" in df.columns:
        df.drop("isnull", axis = 1, inplace = True)
    df.drop(df.columns[["Unnamed" in x for x in df.columns]], axis = 1, inplace = True)
    tm = df.columns[["tm" in x for x in df.columns]][0]
    df[tm] = df["yyyymmdd"].apply(lambda x: str(x)[:4] + "-" + str(x)[4:6] + "-" + str(x)[6:])
    idx = df.columns[[x not in ["yyyymmdd", tm, "stn_id"] for x in df.columns]]
    print(idx)
    df = pd.merge(df, area, how = "left", on = "stn_id")
    for i in df[df["name"].isnull()]["stn_id"].unique():
        #step 1 :과반수 이상의 열에서 해당 관측소 결측 50% 이상
        #if (np.round(df.query("stn_id == @i")[idx].isnull().mean(), 1) > 0.5).mean() > 0.5:
        #    df = df.query("stn_id != @i").reset_index(drop = True)
        #step 2 : 필요한 기간의 데이터가 없을 경우(특히 타겟시점!)
        un = df.query("stn_id == @i").dropna(subset = idx)[tm].apply(lambda x: x[:4]).unique()
        if "2011" in un and len(un) < 6:
            df = df.query("stn_id != @i").reset_index(drop = True)
        elif len(un) < 4 or "2016" not in un:
            df = df.query("stn_id != @i").reset_index(drop = True)
    if to_csv == True:
        df.to_csv(dfname + ".csv", encoding = "utf-8-sig", index = False)
    return idx, df

In [11]:
df = preprocess(df)
print(df.isnull().sum())
df

tma              0
stn_id           0
avg_pa         117
avg_ps           0
max_ps           3
max_ps_hrmt      3
min_ps           0
min_ps_hrmt      3
yyyymmdd         0
dtype: int64


,tma,stn_id,avg_pa,avg_ps,max_ps,max_ps_hrmt,min_ps,min_ps_hrmt,yyyymmdd
0,2011-01-01 00:00:00.0,79,999.8,1020.9,1026.6,2400.0,1016.5,100.0,20110101
1,2011-01-01 00:00:00.0,80,-99.9,0.0,-99.9,-999.0,9999.9,-999.0,20110101
2,2011-01-01 00:00:00.0,81,1018.5,1021.0,1025.9,2300.0,1016.9,200.0,20110101
3,2011-01-02 00:00:00.0,79,1005.0,1026.1,1028.4,800.0,1024.3,1500.0,20110102
4,2011-01-02 00:00:00.0,80,-99.9,0.0,-99.9,-999.0,9999.9,-999.0,20110102
...,...,...,...,...,...,...,...,...,...
10529,2016-03-27 00:00:00.0,19,NaN,0.0,1014.2,100.0,1013.7,200.0,20160327
10530,2016-03-28 00:00:00.0,19,NaN,0.0,1014.3,100.0,1014.3,100.0,20160328
10531,2016-03-29 00:00:00.0,19,NaN,0.0,1015.7,100.0,1015.6,200.0,20160329
10532,2016-03-30 00:00:00.0,19,NaN,0.0,1012.3,200.0,1012.1,100.0,20160330


In [12]:
df = df.loc[:, ["tma", "stn_id", "avg_pa", "yyyymmdd"]]

In [13]:
back_path = "/Users/seong-yeseul/Downloads/pm10_agg_mac.csv" #경로는 알아서,,
back = pd.DataFrame(pd.read_csv(back_path, encoding = "utf-8-sig")["yyyymmdd"].unique(), columns = ["yyyymmdd"])
df = daily(df, back)
print(df.isnull().sum())
df

79  is processing
80  is processing
81  is processing
82  is processing
83  is processing
84  is processing
85  is processing
86  is processing
87  is processing
89  is processing
59  is processing
35  is processing
2  is processing
4  is processing
16  is processing
17  is processing
18  is processing
19  is processing
20  is processing
27  is processing
28  is processing
29  is processing
36  is processing
37  is processing
38  is processing
39  is processing
40  is processing
42  is processing
47  is processing
48  is processing
49  is processing
53  is processing
58  is processing
62  is processing
63  is processing
64  is processing
70  is processing
72  is processing
74  is processing
233  is processing
129  is processing
yyyymmdd        0
tma         79338
stn_id          0
avg_pa      79455
dtype: int64


,yyyymmdd,tma,stn_id,avg_pa
0,20110101,2011-01-01 00:00:00.0,79,999.8
1,20110102,2011-01-02 00:00:00.0,79,1005.0
2,20110103,2011-01-03 00:00:00.0,79,1001.1
3,20110104,2011-01-04 00:00:00.0,79,1001.0
4,20110105,2011-01-05 00:00:00.0,79,1000.5
...,...,...,...,...
2187,20161227,NaN,129,NaN
2188,20161228,NaN,129,NaN
2189,20161229,NaN,129,NaN
2190,20161230,NaN,129,NaN


# Mapping

In [3]:
area = pd.read_csv("mapping_0713.csv")
df = pd.read_csv("daily_sfc_prsr.csv", encoding = "utf-8-sig")
display(df.head())
print(df.isnull().mean()) #tma결측인 이유 : yyyymmdd기준 합병했기 때문에 해당 데이터가 원래 없으면 나머지 싹 결측치로 들어감.

,yyyymmdd,tma,stn_id,avg_ps,max_ps,min_ps
0,20110101,2011-01-01,90,1021.3,1026.6,1013.5
1,20110102,2011-01-02,90,1026.4,1028.2,1024.7
2,20110103,2011-01-03,90,1023.0,1025.8,1020.5
3,20110104,2011-01-04,90,1018.9,1020.5,1017.7
4,20110105,2011-01-05,90,1017.0,1019.3,1014.7


yyyymmdd    0.000000
tma         0.058984
stn_id      0.000000
avg_ps      0.059689
max_ps      0.059182
min_ps      0.059182
dtype: float64


In [6]:
idx, df = after_daily(df, area)

Index(['avg_ps', 'max_ps', 'min_ps'], dtype='object')


In [7]:
df.isnull().sum()

yyyymmdd        0
tma             0
stn_id          0
avg_ps      10770
max_ps      10660
min_ps      10660
name            0
address         0
lat             0
long            0
add             0
dtype: int64

In [8]:
for col in idx:
    print(col)
    ii = df[df[col] < 500].index
    if len(ii) > 0:
        display(df.loc[ii, :])
        #df.loc[ii, col] = np.nan
    ii = df[df[col] > 1080].index
    if len(ii) > 0:
        display(df.loc[ii, :])
        #df.loc[ii, col] = np.nan
#최저 500, 최고 1080 벗어나는 범위 없음!

avg_ps
max_ps
min_ps


In [9]:
df.to_csv("null_sfc_prsr.csv", encoding = "utf-8-sig", index = False)

# 결측치처리 : 거리기준 단순평균

In [ ]:
area3 = df[["stn_id", "name", "lat", "long", "add"]].drop_duplicates().reset_index(drop = True)
for col in idx:
    for s in df[df[col].isnull()]["stn_id"].unique():
        latlon = df.query("stn_id == @s")[["lat", "long"]].drop_duplicates().values[0]
        x = (np.cos(latlon[0]) * 6400 * 2 * np.pi / 360) * np.abs(latlon[1] - area3[["long"]])
        y = 111 * np.abs(latlon[0] - area3["lat"])
        distance = np.sqrt((x**2).squeeze() + y**2).sort_values().head(6)
        dd = pd.concat([area3.iloc[distance.index.to_list(), :], pd.DataFrame(distance, columns = ["dist"])], axis = 1)
        #display(dd)
        tg = df[df["stn_id"] == s]
        ss = dd["stn_id"]
        ii = tg[tg[col].isnull()].index
        ymd = df.loc[ii, "yyyymmdd"]
        #display(df.loc[ii, :])
        df.loc[ii, col] = np.nanmean(df.query("stn_id in @ss and stn_id != @s").pivot(index = "yyyymmdd", columns = "stn_id", values = col).query("yyyymmdd in @ymd"), axis = 1)
        #print(np.nanmean(df.query("stn_id in @ss and stn_id != @s").pivot(index = "yyyymmdd", columns = "stn_id", values = col).query("yyyymmdd in @ymd"), axis = 1))
df.isnull().sum()

In [ ]:
df.to_csv("final_sfc_prsr.csv", encoding = "utf-8-sig", index = False)